## Семинар 6: машинный перевод

На этом семинаре по будем работать с обучением типа "последовательность к последовательности" (seq2seq), когда нам необходимо по данной цепочке слов (символов) предсказать другую цепочку (например, на другом языке):

"the cat sat on the mat" -> [Seq2Seq model] -> "le chat etait assis sur le tapis"

Seq2Seq обучение в Keras будем проводить на уровне символов. Работу модели будем проверять на посимвольном переводе
коротких английских предложения в короткие французские предложения. В промышленных система более распространен перевод на уровне слов (с вложениями слов).

- В качетсве входных последовательностей выступают английские предложения, в качестве соответствующих целевых последовательностей - французские предложения.

- Кодировщик LSTM конвертирует входные последовательности в 2 вектора состояния (нам необходимо последнее состояние LSTM, выходы как таковые не нужны).

- LSTM декодер обучается преобразовывать целевые последовательности в те же последовательности, но смещенные на один шаг вперед. Это назвается "принуждение учителя" (teaching force). Декодер использует в качестве начального состояния векторы состояния из кодировщика. Декодер учится генерировать target[t+1...] по target[...t] при условии входной последовательности.

![Пример 1](https://blog.keras.io/img/seq2seq/seq2seq-teacher-forcing.png)

- В режиме вывода, когда мы хотим декодировать неизвестные входные последовательности:
    - Кодируем входную последовательность в векторы состояний
    - Начнинаем с целевой последовательности размера 1 (символ начала последовательности)
    - Отправляем векторы состояния и целевую последовательности из одного символа
        в декодер для получения предсказаний для следующего символа
    - Генерируем следующий символ, используя эти предсказания (argmax).
    - Добавляем полученный символ в целевую последовательность
    - Повторяем, пока не будет получен символ конца последовательности или не будет достигнут лимит символов.
    
![Пример 2](https://blog.keras.io/img/seq2seq/seq2seq-inference.png)

Данные, которые будем исользовть:

- [English to French sentence pairs.](http://www.manythings.org/anki/fra-eng.zip) - используем этот датасет
- [Lots of neat sentence pairs datasets.](http://www.manythings.org/anki/)

Дополнительные ссылки:
- [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)


In [ ]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'


### Часть 1 - Загрузка данных, создание кодировщика

Векторизуем наши входные и выходные последовательности. Используем one-hot кодирование.

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
### Your code here
# Читаем файл в массив строчек
lines = None

for line in lines[: min(num_samples, len(lines) - 1)]:
    ### Your code here
    # из каждой строчки извлекаем входной текст и целевой
    # [START] - tab, [STOP] - перевод строки
    input_text, target_text = None, None
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    ### Your code here
    # подситываем статистику: уникальные символы во входных последовательностях (input_characters) 
    #и в целевых (target_characters)
    input_characters.add(None)
    target_characters.add(None)

Вывыодим статистическую информацию

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
### Your code here
# Подсчитываем максимальные длины последовательностей для входа (max_encoder_seq_length) и цели (max_decoder_seq_length)
max_encoder_seq_length = None
max_decoder_seq_length = None

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

### Your code here
# Создаем индекс входных символов и выходных: словари типа char:index
input_token_index = dict()
target_token_index = dict()

Создаем массивы для данных: 
- encoder_input_data - входной массив всех входных последовательностей, закодированных one-hot
- decoder_input_data - целевой массив всех целевых последовательностей, закодированных one-hot
- decoder_target_data - целевой массив всех целевых последовательностей, закодированных one-hot, со смещением на 1

In [ ]:
### Your code here
encoder_input_data = np.zeros(
    (0, 0, 0),
    dtype='float32')
decoder_input_data = np.zeros(
    (0,0,0),
    dtype='float32')
decoder_target_data = np.zeros(
    (0,0,0),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    ### Your code here
    # заполняем нулевые массивы 1 в соответствующие места
    pass


Создаем слой LSTM с параметром возвращения внутренних состояний (return_state).

![LSTM](https://i.stack.imgur.com/DS97N.png)

In [ ]:
latent_dim = 256  # Latent dimensionality of the encoding space.

# Define an input sequence and process it.
### Your code here
encoder_inputs = Input(shape=(None, 0))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

### Часть 2 - Создание декодировщика, обучение

Декодером будет являться также LSTM, в которйо в качестве начального состояния будет исопльзоваться выход кодировщика.

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
### Your code here
decoder_inputs = Input(shape=(None, 0))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

# Добавляем полносвязный слой для определения вероятности следующейго символа
decoder_dense = None
decoder_outputs = None

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

### Часть 3 - режим вывода

Используем те же созданные нами ранее слои, но немного меняем порядок подачи данных на декодер.

In [ ]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


Функция декодирование последовательности - перевод в символы.

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
### Your code here
reverse_input_char_index = dict()
reverse_target_char_index = dict(0)

def decode_sequence(input_seq):
    ### Your code here
    # Encode the input as state vectors.
    states_value = None

    
    ### Your code here
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((0,0,0))
    ### Your code here
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        ### Your code here
        # выбираем максимальной правдоподобный
        sampled_token_index = None
        # определяем символ
        sampled_char = None
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        ### Your code here
        # определяем символ
        # Update the target sequence (of length 1).
        target_seq = np.zeros((0, 0, 0))
        target_seq[0, 0, 0] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

Запускаем генерацию

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)